In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('Data/Listening_Data/songs_listened_to_with_all_info_unique.csv')
df = df.drop('Unnamed: 0.1', axis=1)
df = df.drop('Unnamed: 0', axis = 1)
df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,month,year,Season,trackUri,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4


In [32]:
one_hot_df = pd.concat([df.drop('month', axis=1), pd.get_dummies(df['Season'])], axis=1)
one_hot_df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,year,Season,trackUri,danceability,energy,...,id,uri,track_href,analysis_url,duration_ms,time_signature,1,2,3,4
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0


In [33]:
one_hot_df.columns

Index([         'endTime',       'artistName',        'trackName',
              'albumName',         'msPlayed',             'year',
                 'Season',         'trackUri',     'danceability',
                 'energy',              'key',         'loudness',
                   'mode',      'speechiness',     'acousticness',
       'instrumentalness',         'liveness',          'valence',
                  'tempo',             'type',               'id',
                    'uri',       'track_href',     'analysis_url',
            'duration_ms',   'time_signature',                  1,
                        2,                  3,                  4],
      dtype='object')

In [42]:
df['Count'] = 1
count_df = df.groupby('trackName', as_index = False).sum('Count')
count_df[['trackName', 'Count']]

,trackName,Count
0,#1 Freak (feat. Ty Dolla $ign),9
1,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,1
2,1 0 0 . m i l ‘ (with Bas),1
3,1-800-273-8255,2
4,10 F*CKS (with Mansa),7
...,...,...
1845,upsides,11
1846,viola,1
1847,way back,7
1848,when the water meets the mountains (acoustic v...,2


,Count
0,73
1,73
2,73
3,73
4,73
...,...
14121,3
14122,3
14123,3
14124,1


In [54]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets

features = one_hot_df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 1, 2, 3, 4]]
labels = pd.merge(one_hot_df, count_df, on='trackName')[['Count']]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [66]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(features, labels)


/Users/jamesyang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/var/folders/h4/6452mjq15618sb_49rqm1mkw0000gn/T/ipykernel_68980/2590962362.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(features, labels)


RandomForestRegressor(n_estimators=1000, random_state=42)

In [95]:
# Get the new song
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

cid = '7cda6c1856a24d109ba5521fd35fd5a3'
secret = 'ac6d7cd270aa48cab17e179262d78031'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

new_song_uri = 'spotify:track:4b3rGL8LDSkbARUUA20136'
song_stats = pd.DataFrame.from_dict(sp.audio_features(new_song_uri)).drop(['type', 'mode', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature', 'duration_ms'], axis = 1)
song_stats

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.812,0.807,9,-3.354,0.339,0.11,0,0.211,0.4,136.979


In [96]:
# Attach the Season
song_stats[1] = 1
song_stats[2] = 0
song_stats[3] = 0
song_stats[4] = 0
song_stats

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,1,2,3,4
0,0.812,0.807,9,-3.354,0.339,0.11,0,0.211,0.4,136.979,1,0,0,0


In [97]:
features.head(1)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,1,2,3,4
0,0.566,0.693,10,-6.917,0.368,0.361,0.0,0.0806,0.838,78.743,1,0,0,0


In [98]:
import numpy as np
# Use the forest's predict method on the test data

predictions = rf.predict(song_stats)
# test_labels = test_labels['Count']

/Users/jamesyang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [99]:
# This is how many times you are predicted to listen to it
print("You will listen to " + "Venom by Stray Kids " + str(round(predictions[0])) + " Times")

You will listen to Venom by Stray Kids 15 Times
